<a href="https://colab.research.google.com/github/RiteshopShrivastava/Gold-Price-forecasting/blob/main/Gold_Prices_FbProphet_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 10.4 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date
import plotly.express as px
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

In [4]:
start='2018-01-01'
end=date.today().strftime('%Y-%m-%d')
data=yf.download('GC=F',start=start,end=end)[['Adj Close']]

[*********************100%***********************]  1 of 1 completed


In [5]:
data.tail()

,Adj Close
Date,
2021-12-21,1787.900024
2021-12-22,1801.599976
2021-12-23,1811.199951
2021-12-27,1808.099976
2021-12-28,1810.199951


In [6]:
training_size=data.shape[0]-30
training= data[:training_size]
test=data[training_size:]

In [14]:
m=Prophet()

In [8]:
training.reset_index(inplace=True)

In [9]:
training.dtypes

Date         datetime64[ns]
Adj Close           float64
dtype: object

In [10]:
training.columns=['ds','y']

In [12]:
training

,ds,y
0,2018-01-02,1313.699951
1,2018-01-03,1316.199951
2,2018-01-04,1319.400024
3,2018-01-05,1320.300049
4,2018-01-08,1318.599976
...,...,...
970,2021-11-09,1830.199951
971,2021-11-10,1847.599976
972,2021-11-11,1863.199951
973,2021-11-12,1867.900024


In [15]:
m.fit(training)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


### **Model Validation**

In [16]:
future_prices=m.make_future_dataframe(periods=30)

In [17]:
pred=m.predict(future_prices)

In [18]:
pred[['ds','yhat','yhat_lower','yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1000,2021-12-11,1752.456412,1714.426551,1795.372902
1001,2021-12-12,1754.970249,1713.288216,1796.127930
1002,2021-12-13,1759.996469,1718.918114,1797.334788
1003,2021-12-14,1762.554891,1722.350895,1808.190004
1004,2021-12-15,1765.258187,1724.477821,1808.349768


In [19]:
prediction_data=pred[['ds','yhat']]
pred=prediction_data[prediction_data.shape[0]-30:]

In [24]:
pred

,ds,yhat
975,2021-11-16,1768.279498
976,2021-11-17,1765.917977
977,2021-11-18,1764.054862
978,2021-11-19,1760.873394
979,2021-11-20,1757.322446
980,2021-11-21,1755.137943
981,2021-11-22,1755.368946
982,2021-11-23,1753.094267
983,2021-11-24,1750.987500
984,2021-11-25,1749.555548


In [22]:
y_pred=pred['yhat'].values
y_test=test['Adj Close'].values

In [23]:
y_pred

array([1768.27949763, 1765.91797694, 1764.0548617 , 1760.87339435,
       1757.32244637, 1755.13794287, 1755.36894637, 1753.09426654,
       1750.98750024, 1749.55554842, 1746.98704955, 1744.23330729,
       1743.0294651 , 1744.42041279, 1743.47729932, 1742.86256147,
       1743.06851805, 1742.26596948, 1741.38537865, 1742.13838107,
       1745.54410226, 1746.64614754, 1748.0781626 , 1750.30300161,
       1751.46192853, 1752.45641234, 1754.97024855, 1759.9964692 ,
       1762.55489119, 1765.25818669])

In [21]:
rmse=np.round(np.sqrt(mean_squared_error(y_test,y_pred)),2)

In [ ]:
print('RMSE of the data is : {}'.format(rmse))

RMSE of the data is : 49.75


### **Forecasting for next 30 days**

In [84]:
import datetime
from datetime import date, timedelta, datetime
from pandas import DataFrame
from pandas import to_datetime
future = list()
date= datetime.now()
for i in range(0, 30):
  date= date+ timedelta(days=1)
  future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds'] = to_datetime(future['ds'])

In [85]:
forecast = m.predict(future)

In [86]:
forecast[['ds','yhat']]

,ds,yhat
0,2021-12-30 20:35:35.468086,1799.069167
1,2021-12-31 20:35:35.468086,1798.332358
2,2022-01-01 20:35:35.468086,1798.275013
3,2022-01-02 20:35:35.468086,1800.777564
4,2022-01-03 20:35:35.468086,1800.970335
5,2022-01-04 20:35:35.468086,1800.389129
6,2022-01-05 20:35:35.468086,1800.622004
7,2022-01-06 20:35:35.468086,1799.398103
8,2022-01-07 20:35:35.468086,1797.631974
9,2022-01-08 20:35:35.468086,1796.668633
